복잡한 공유한 상태를 없애고 엄격하지 않은 처리를 위주로 설계를 진행한다면, 성능을 향상시키기 위해 병렬성을 활용할 수 있다. 특히, 지연 계산을 허용하는 알고리즘을 적용하는 경우, 파이썬 라이브러리 패키지가 도움이 된다.

핵심 아이디어는 함수형 프로그램을 한 프로세스 안의 여러 스레드나 여러 프로세스에 분배하는 것이다. 제대로된 함수형 설계를 만들었다면 애플리케이션의 각 부분 사이에 복잡한 상호 작용이 없을 것이다. 단지 인자 값을 받고 결과를 만들어내는 여러 함수들만 존재한다. 프로세스나 스레드의 경우 이러한 구조가 가장 이상적이다.

multiprocessing과 concurrent.futures 모듈에 집중할 것이다. 이 두 모듈은 몇 가지 병렬 처리 기법을 제공한다. 

또한 다중 스레드 대신 프로세스 수준의 병렬성에 집중할 것이다. 프로세스 수준의 병렬성에 집중한다면 파이썬의 GIL을 신경 쓰지 않고 상당한 성능 향상을 달성할 수 있다. 

threading 모듈의 여러 기능은 강조하지 않을 것이다. 이 모듈을 사용해 병렬 처리를 하는 경우도 종종 있다. 함수형 프로그래밍 설계를 제대로 했다면, 여러 스레드에서 같은 메모리를 쓰기 때문에 생기는 문제를 최소화할 수 있다. 하지만 GIL이 있기 때문에 CPython에서 다중 스레드를 사용하는 애플리케이션에는 몇 가지 제한이 있다. I/O를 기다리는 경우에는 GIL에 걸리지 않기 때문에 I/O 위주의 프로그램은 다중 스레드를 사용해도 상당히 좋은 결과를 얻을 수 있다.

수행하는 작업 사이에 의존 관계가 없다면 가장 효율적인 병렬 처리가 가능하다. 충분히 주의 깊게 설계한다면, 병렬 프로그래밍을 이상적인 처리 기법으로 활용할 수 있다. 병렬 프로그램을 작성할 때 가장 어려운 부분은 공유 자원의 갱신을 조정하는 것이다.

함수형 설계 패턴을 따르고 상태가 있는 프로그램을 피한다면, 공유한 객체를 동시에 갱신하는 것도 최소화할 수 있다. 엄격하지 않은 지연 계산ㄴ을 위주로 하는 소프트웨어를 설계할 수 있다면, 동시 계산이 가능한 소프트웨어도 설계할 수 있다. 

프로그램에서 연산의 순서가 문제가 되는 경우라면 언제나 엄격한 의존 관계가 존재한다. 예를 들어, 2*(3+a)라는 식에서는 (3+a)라는 하위 식을 먼저 계산해야 한다. 하지만 컬레겻능ㄹ 다루는 경우라면, 컬렉션 내의 각 원소를 처리하는 순서가 중요하지 않은 상황과 자주 마주칠 수 있다. 

다음 두 예제를 생각해보자.

In [2]:
y = [1, 2, 3, 4, 5, 6, 7, 8]
def func(x):
    return x*x

x = list(func(item) for item in y)

In [3]:
x

[1, 4, 9, 16, 25, 36, 49, 64]

In [4]:
x = list(reversed([func(item) for item in y[::-1]]))

In [5]:
x

[1, 4, 9, 16, 25, 36, 49, 64]

두 명령에서 원소를 평가하는 순서는 서로 반대지만, 결과는 동일하다. 심지어는 다음과 같은 명령도 같은 결과를 만들 수 있다.

In [7]:
import random
indices = list(range(len(y)))
random.shuffle(indices)
x = [None]*len(y)
for k in indices:
    x[k] = func(y[k])

In [8]:
x

[1, 4, 9, 16, 25, 36, 49, 64]

임의의 순서로 원소를 평가했다. 각 원소를 평가하는 것이 서로 독립적이기 때문에 그 순서는 문제가 되지 않는다. 엄격하지 않은 계산을 허용하는 많은 알고리즘도 이와 비슷한 경우다.

### 동시성의 진정한 의미는 무엇인가?

단일 코어 프로세서가 하나만 있는 작은 컴퓨터에서는 모든 계산이 그 유일한 프로세서 코어에 의해 직렬화된다. 운영체제는 시간을 영리하게 분할해 할당함으로써 여러 프로세스나 여러 스레드를 서로 뒤섞여 처리해준다.

CPU가 여러 개 있거나, CPU는 하나지만 코어가 여럿인 컴퓨터의 경우에는 진짜로 동시에 CPU 명령을 처리할 수 있다. 다른 동시성은 OS수준의 시분할을 통해 시뮬레이션되는 것이다. 맥 OS X 데스크톱에는 CPU를 공유하는 프로세스가 200개 있을 수 있다. 이는 사용 가능한 코어 개수를 훨씬 넘어선다. 이를 생각해볼 때 겉으로는 동시적인 것처럼 보이는 동작의 대부분은 OS 수준의 시분할에서 비롯된 것임을 알 수 있다.

## 경계 조건

$O(n^2)$ 복잡도 알고리즘이 있다고 가정해보자. 1,000바이트의 파이썬 코드로 이뤄진 내부 루프가 있다고 하자. 10,000개의 객체를 처리한다면 100조 바이트의 파이썬 연산을 실행해야 한다. 이것이 예상 처리 비용이다. 도움이 되리라 생각하는 스레드나 프로세스를 얼마든지 할당할 수는 있지만, 전체 예상 처리 비요은 변하지 않는다. 

CPython 바이트코드 각각의 실행 시간은 간단ㄴ하지 않다. 하지막 맥 OS X 노트북에서 장시간에 걸쳐 평균을 내 본 결과로는 초당 60MB정도의 바이트코드를 실행할 것으로 예상할 수 있다. 이는 앞에서 예로 든 100조 바이트의 파이썬 연산을 처리하려면 28분이 걸릴 것이라는 의미다. 

코어가 4개인 듀얼 프로세서 컴퓨터가 있다면 예상 시간을 25퍼센트, 즉 7분으로 줄일 수 있다. 이 계산에는 우리가 작업을 4개의 독립적인 OS 프로세스로 나눌 수 있다는 가정이 들어 있다.

여기서 중요하게 고려해야 할 점은 100조 바이트의 바이트코드라는 예상 처리 비용은 달라지지 않았다는 점이다. 병렬화가 무슨 마법처럼 부하를 줄여주지는 못한다. 다만 전체 실행 시간을 줄여줄 수 있도록 처리를 스케줄링해줄 뿐이다.

$O(n log n)$ 복잡도의 더 좋은 알고리즘을 패택한다면 예상 처리 비용을 132MB의 바이트코그 연산으로 바꿀 수 있다. 60MB/s로 실행한다면, 이 정도의 부하는 상당히 작은 것이다. 병렬화는 알고리즘의 개선이 가져오는 것만큼 극적인 향상을 가져오지는 못한다.

### 프로세스나 스레드 사이에 자원 공유하기

운영체제는 프로세스 간의 상호작용이 없거나 거의 없다고 확신한다. 상호작용하는 두 프로세스가 있다면, 몇 가지 일반적인 OS 자원을 명시적으로 공유해야만 한다. 그러한 자원에는 공통의 파일이나 특정 공유 메모리 객체, 또는 프로세스 사이에 상태를 공유하는 세마포어 등이 있다. 프로세스는 근본적으로 독립적인 존재들이며, 프로세스 간의 상호 작용은 예외적인 상황이다.

반면, 다중 스레드는 단일 프로세스의 일부다. 한 프로세스에 속한 모든 스레드는 같은 OS 자원을 공유한다. 다른 스레드에 영향을 주지 않고 자유롭게 쓸 수 있는 스레드-지역(thread-local) 변수는 이러한 공유의 예외다. 스레드-지역 메모리를 벗어난 메모리에 대한 쓰기 연산을 수행하면 프로세스의 내부 상태를 예측할 수 없는 순서로 메모리가 변경될 수도 있다. 이렇게 상태 변경을 수행할 때 생기는 문제를 피하려면 락(lock)을 명시적으로 사용해야 한다. 앞에서 설명한 것처럼, 명령어 시행 시퀀스가 모두 동시에 이뤄지는 경우는 극히 드물다. 동시에 실행되는 여러 스레드나 프로세스의 명령어들은 일반적으로 미리 예측할 수 없는 순서로 서로 뒤섞인다. 스레드를 사용하는 경우, 공유 변수의 내용을 덮어 쓸 가능성이 있기 때문에 주의깊게 사용해야 한다. 병렬 프로세스를 사용하는 경우, 운영체제 수준의 프로세스 스케줄링이라는 부가 비용이 든다. 

실제로는, 심지어 하드웨어 수준에서도 복잡한 메모리 쓰기 상황이 발생한다. 이러한 메모리 쓰기 상황에 대해서는 http://en.wikipedia.org/wiki/Memory_disambiguation를 참조하자.

동시에 객체를 갱신하는 경우가 있기 때문에 다중 스레드 애플리케이션을 설계하려고 할 때 혼란이 생긴다. 록을 사용하는 것은 공유 객체에 대한 동시 쓰기를 방지하는 한 가지 방법이다. 공유 객체를 피하는 것도 실현 가능성이 있는 설계 기법 중 하나이다. 이러한 기법은 함수형 프로그래밍에 더 적용하기 쉽다.

C 파이썬은 GIL을 사용해 OS 스레드 스케줄링이 파이썬 데이터 구조 갱신에 영향을 끼치지 못하도록 보장한다. 그 결과, GIL이 기계가 제공하는 명령어 수준에서 파이썬 가장 머신의 명령어 수준으로 스케줄링의 기본 단위를 바꿔준다. GIL이 없다면 서로 경쟁 관계에 있는 여러 스레드가 서로 뒤섞여 실행되면서 파이썬의 내부 데이터 구조를 어염시킬 가능성이 있다.

### 어디서 이익이 누적되는가?

수행하는 계산량이 많은 반면, 상대적으로 I/O가 적은 프로그램은 동시 프로세싱을 채택해도 그리 큰 이익을 볼 수가 없다. 예상 처리 비용이 28분인 계산이 있다면, 연산을 다른 방식으로 배열한다고 해도 결과에 그렇게 큰 영향이 없을 것이다. 100조 바이트의 코드를 엄격한 계산에서 엄격하지 않은 계산으로 바꾼다고 해도 전체 실행 시간이 줄어들지는 않을 것이다. 

하지만 계산에 I/O가 많이 들어가거나 CPU 처리와 I/O 요청을 서로 엇갈려 배치하면 성능이 좋아질 수 있다. 이상적인 경우, 프로그램이 데이터를 처리하는 동안 운영 체제가 다음 번 데이터를 기다릴 수 있다면 좋을 것이다.

계산과 I/O를 서로 섞여 진행하는 데는 두 가지 접근 방식이 있다. 각각은 다음과 같다.

* I/O와 문제에 대한 계산을 전체적으로 엇갈려 진행할 수 있다. 일기, 처리, 쓰기를 수행하는 처리 파이프라인을 만들 수 있을 것이다. 데이터 객체가 그 파이프의 한 단계에서 다음 단계로 계속 흘러가게 만드는 것이 아이디어다. 각 단계는 병렬적으로 진행할 수 있다.

* 문제를 각각 시작부터 끝까지 처리할 수 있는 별도의 독립적인 단위로 더 작게 분할할 수도 있다.

이러한 두 가지 접근 방식의 경계가 명확한 것은 아니다. 두 방식의 중간에 겹친 영역이 존재한다. 예를 들어, 여러 병렬 파이프라인을 사용하는 것은 두 설계를 혼합한 기법이다. 동시성 프로그램 설계를 어느 정도 쉽게 만들어주는 형식화 기법이 존재한다. 순차 프로세스(Communicating Sequential Process)라는 패러다임은 메시지 전달 애플리케이션의 설계에 도움이 된다. pycsp와 같은 패키지를 사용하면 파이썬에 CSP기법을 도입할 수 있다.

I/O위주의 프로그램은 동시 프로세싱에서 이익을 얻을 수 있다. 핵심은 I/O와 처리를 교대로 수행하는 것이다. CPU 위주의 프로그램은 동시 프로세싱을 사용하여 이익을 보는 경우가 드물다.

### 다중 프로세싱 풀과 작업 사용하기

엄격하지 않은 평가를 더 큰 맥락에서 사용하기 위해 multiprocessing 패키지에 pool 객체라는 개념을 도입했다. 동시 작업 프로세스로 이뤄진 Pool 객체를 만들고, 작업을 각 프로세스에 할당한 후 작업이 도잇에 실행될 것을 기대할 수 있다. 앞에서 설명한 것 처럼, 이렇게 생성했다고 하여 Pool 안의 모든 객체가 한꺼번에 만들어지는 것은 아니다. 이는 운영체제의 스케줄링이 여러 프로세스의 실행을 서로 뒤섞을 수 있기 때문에 실행 순서를 예측하기는 어렵다는 의미이기도 하다. 일부 애플리케이션은 이런 방법을 사용하여 더 빠른 시간에 더 많은 작업을 수행할 수 있는 경우도 있다.

이러한 기능을 가장 잘 활용하려면 애플리케이션을 엄격하지 않은 동시 실행에서 이익을 얻을 수 있는 여러 부분으로 나눠야 한다. 임의의 순서로 실행될 수 있는 여러 작업을 별도로 정의할 수 있으면 좋다. 

인터넷에서 웹을 긁어와 데이터를 얻는 애플리케이션을 병렬 처리를 사용해 쉽게 최적화할 수 있다. 동일한 웹 사이트 수집기가 여러 개 들어 있는 Pool 객체를 만들 수 있다. 풀에 있는 프로세스들이 처리할 작업은 URL에서 데이터를 가져와 분석하는 석이다.

여러 로그 파일을 분석하는 애플리케이션도 병렬화하면 좋은 대상일 것이다. 분석 프로세스로 이뤄진 Pool을 만들 수 있다. 분석기마다 각각의 로그 파일을 할당하면, 풀에 있는 여러 작업 프로세스가 실행하는 각 분석기가 동시에 파일을 읽고 분석할 수 있다. 각각의 작업 프로세스는 직렬화한 I/O와 계산으로 이뤄진다. 하지만 한 작업 프로세스가 I/O 완료를 기다리는 동안 다른 작업 프로세스는 분석 계산을 수행할 수 있다. 

### 여러 큰 파일 처리하기

다중 프로세스 애플리케이션의 예를 살펴보자. 웹 로그 파일에 있는 일반 로그 형식 데이터를 분석할 것이다. 이 형식은 접근 로그에 일반적으로 사용하는 양식이다.

때로 매우 크기가 큰 로그 파일을 매우 많이 분석해야 할 수도 있다. 독립적인 파일이 많이 존재한다는 것은 동시성을 활용하면 상당한 성능 향상을 얻을 수 있다는 의미다.

분석을 넓게 볼 때 두 가지 기능으로 나눌 수 있다. 모든 처리의 첫 단계는 로그 파일을 구문 분석하여 관심 대상 정보를 가져오는 것이다. 이를 다시 네 단계로 나눌 수 있다. 각각은 다음과 같다.

1. 여러 로그 파일의 모든 줄을 읽는다.
2. 파일의 컬렉션에 있는 각 줄의 로그 항목으로부터 이름 있는 튜플을 만든다.
3. 날짜나 URL과 같이 더 복잡한 필드를 자세히 분석한다.
4. 로그에서 관심의 대상이 아닌 경로를 제거한다. 이를 관심 대상인 경로만을 받아들이는 것으로 생각할 수도 있다.

이러한 구문 분석 단계를 거치고 나면, 다양한 분석이 가능하다. multiprocessing 모듈의 기능을 보여주기 위해 특정 경로에 대한 요청이 들어온 회수를 세는 단순한 분석을 살펴본다.

대부분의 입력 처리는 원본 파일을 읽어오는 첫 번째 부분으로 이뤄진다. 파이썬에서 파일 반복자를 사용하면 저수준의 운영체제 요청으로 바뀐다. 각 운영체제 요청이 의미하는 것은 데이터가 사용할 수 있을 때까지 프로세스가 기다려야만 한다는 것이다.

분명히 그러한 경우 다른 연산을 끼워 넣더 I/O가 완료될 때까지 기다리지 않고 처리하기를 원할 것이다. 개발 행부터 전체 파일에 이르기까지 다양한 범우의 연산을 I/O 사이에 끼워넣을 수 있다. 상대적으로 구현하기 쉽기 때문에 전체 파일을 대상으로 하는 연산을 끼워넣는 것을 먼저 살펴본다.

아파치 CLF 파일을 구문 분석하는 것을 함수형으로 설계하면 다음과 같다.

data = path_filter(access_detail_iter(access_iter(local_gzip(filname))))

큰 구문 분석 문제를 각각의 단계를 처리하는 작음 함수를 여러 개 사용해 해결했다. local_gzip() 함수는 지역적으로 캐시된 GZIP 파일의 각 줄을 읽는다. acess_iter 함수는 접근 로그의 각 줄을 가지고 이름 있는 튜플을 만든다. access_detail_iter 함수는 분석하기 어려운 필드를 확장한다. path_filter 함수는 분석 시 가치가 적은 일부 경로와 파일 확장자를 제거한다.

### 로그 파일 구문 분석하기 - 행 수집하기 

각 파일을 읽어 행의 시퀀스를 만들어 내는 것은 여러 파일을 구문 분석하는 첫 번째 단계다. 로그 파일이 .gzip 형식이기 때문에 각 파일을 \__builtons__.open() 함수나 io.open() 함수가 아니라 gzip.open() 함수를 사용해 열어야 한다.

local_gzip() 함수는 지역적으로 캐시한 파일의 각 줄을 읽는다. 다음 코드를 살펴보자.























In [10]:
def local_gzip(pattern):
    zip_logs = glob.glob(pattern)
    for zip_file in zip_logs:
        with gzip.open(zip_file, "rb") as log:
            yield (line.decode('us-ascii').rstrip() for line in log)

앞의 함수는 모든 파일에 대해 루프를 수행한다. 각 파일에 대해 만들어지는 값은 각 파일에 들어 있는 각 줄에 대해 반복할 수 있는 제너레이터 함수다. 여기서 와일드카드를 사용한 파일 찾거나 .zip으로 압축된 로그 파일을 열거나 파일을 여러 줄로 나누면서 각 줄 끝의 \n 문자들을 삭제하는 등의 몇 가지 요소를 캡슐화했다.

여기서 필수적인 디자인 패턴은 각 파일에 대한 제너레이터 식을 값으로 만들어 내는 것이다. 앞의 함수를 함수 하나와 그 함수를 각 파일에 적용하는 매핑으로 다시 표현할 수 있다.

비슷한 출력을 만드는 방법이 몇 가지 더 있다. 예를 들어, 다음은 앞에서 본 예제의 내부 for 루프를 대치할 수 있는 다른 버전이다. line_iter() 함수는 주어진 파일의 각 줄을 내보낼 것이다. 

In [11]:
def line_iter(zip_file):
    log = gzip.open(zip_file, "rb")
    return (line.decode('us-ascii').rstrip() for line in log)

line_iter 함수는 gzip.open() 함수를 적용한 후 각 줄을 정리한다. 특정 패턴과 일치하는 파일에 대해 line_iter() 함수를 적용하는 매핑을 다음과 같이 사용할 수 있다.

In [14]:
import glob

map(line_iter, glob.glob(pattern))

이러한 방식의 매핑이 더 간결하기는 하지만 더 이상 참조하는 변수가 없어서 쓰레시 수집이 될 때까지 열린 파일 객체를 남겨두게 된다는 단점이 있다. 처리할 파일의 개수가 많다면 이러한 부분도 무시못할 부가 비용이 된다. 따라서 앞에서 본 local_gzip() 함수에 초점을 맞출 것이다.

앞에서 본 매핑 방식에는 multiprocessing 모듈의 작동 방식에 잘 들어맞는다는 장점이 있다. 작업자 풀을 만들고 작업(파일 읽기 등)을 프로세스 풀에 매핑할 수 있다. 그렇게 하면 파일을 병렬로 읽게 된다. 이때 열린 파일은 별도의 프로세스에 속할 것이다.

이러한 설계의 확장으로는 FTP를 사용해 웹 호스트로부터 파일을 가져오는 두 번째 함수를 포함시키는 것이 있다. 파일을 웹 서버에서 가져오기 때문에 local_gzip() 함수를 사용해 분석할 수 있다.

access_iter() 함수는 local_gzip() 함수의 결과를 사용해 원본 파일의 각 줄에 있는 파일 접근을 표현하는 항목으로부터 이름 있는 튜플을 만든다.

### 로그 항목을 이름 있는 튜플로 만들기

각 로그파일의 각 줄을 모두 읽었다면, 각 줄이 표현하는 세부 사항을 뽑아올 수 있다. 정규식을 사용하면 각 줄을 분해할 수 있다. 그리고 그 결과를 사용해 namedtuple 객체를 만들 수 있다.

다음은 CLF 파일의 각 줄을 구문 부석하는 정규식이다. 

In [18]:
import re
format_pat = re.compile(
    r"(?P<host>[\d\.]+)\s+"
    r"(?P<identity>\S+)\s+"
    r"(?P<user>\S+)\s+"
    r"\[(?P<time>.+?)\]\s+"
    r'"(?P<request>.+?)"\s+'
    r"(?P<status>\d+)\s+"
    r"(?P<bytes>\S+)\s+"
    r'"(?P<referer>.*?)"\s+'
    r'"(?P<user_agent>.+?)"\s*'
    )


이 정규식을 사용해 각 줄을 아홉 가지 데이터 요소로 분해할 수 있다. []dhk "를 사용하여, time, request, referer, user_agent와 같은 복잡한 요소를 쉽게 이름있는 튜플로 만들 수 있다.

각각의 접근 정보를 다음과 같이 namedtuple() 함수를 사용해 요약할 수 있다.

In [21]:
from collections import namedtuple
Access = namedtuple('Access', ['host', 'identity', 'user', 'time', 'request', 'status', 'bytes', 'referrer', 'user_agent'])

다음은 각 파일에서 각 줄을 돌려주는 반복자를 인자로 받는 access_iter() 함수다.

In [22]:
def access_iter(source_iter):
    for log in source_iter:
        for line in log:
            match = format_pat.match(line)
            if match:
                yield Access(**match.groupdict())

local_gzip() 함수의 출력은 시퀀스의 시퀀스다. 바깥쪽 시퀀스는 각가의 로그 파일들로 이뤄져 있다. 각 차일에 대해, 그 파일에 있는 여러 줄을 돌려주는 반복 가능한 시퀀스가 존재하나. 어떤 줄이 패턴과 일치한다면, 그 줄을 파일에 대한 접근 정보다. 따라서 match 딕셔너리로부터 이름이 붙은 튜플인 Access를 만들 수 있다.

여기서 핵심 디자인 패턴은 구문 분석 함수로부터 정적인 객체를 만들어 내는 것이다. 

이 작업을 수행하는 방식도 여러 가지가 있다. 예를 들어, map() 함수를 다음과 같이 사용할 수도 있다.

In [23]:
def access_builder(line):
    match = format_pat(line)
    if match:
        return Access(**match.groupdic())

이 함수는 꼭 필요한 구문 분석만을 수행하고 Access 객체를 만들어 낸다. 이 함수는 Access 객체를 반환하거나 None을 반환할 것이다. 이 부부분이 정규식과 일치하지 않는 줄을 모두 제거하는 것은 앞에서 본 예제와 다른 점이다.

다음은 이 함수를 사용해 여러 로그 파일을 Access 객체의 단일 스트림으로 만드는 방법을 보여준다.

In [26]:
map(access_builder, (line for log in source_iter for line in log))

이 코드는 local_gzip() 함수의 출력을 Access 인스턴스의 시퀀스로 변환한다. 여러 파일로 이뤄진 컬렉션으로부터 읽어온 결과인 반복 가능 객체에 내포된 반복자에 대해 access_builder 함수를 적용했다. 

### Access 객체의 추가 필드 구문 분석하기

앞에서 만들어 낸 Access 객체는 접근 로그의 각 줄에 있는 아홉 가지 필드의 내부 요소를 분석하지 않은 상태다. 각각의 필드를 더 고수준의 필드들로 분해할 것이다. 그러한 작업을 더 간단한 구문 분석 연산으로 나눈다면 좀 더 단순한 정규식을 사용할 수 있다.

결과 객체는 원래의 Access 튜플을 감싸는 namedtuple 객체다. 그 안에는 자세한 내용을 별도로 구문 분석한 필드가 들어간다.

In [27]:
AccessDetails = namedtuple('AccessDetails', ['access', 'time', 'method', 'url', 'protocol', 'referrer', 'agent'])

다음은 에이전트 상세 정보를 표현하는 필드다.

In [28]:
AgentDetail = namedtuple('AgentDetails', ['product', 'system', 'platform_details_extensions'])

이러한 필드는 대부분의 에이전트 설명이 따르는 가장 일반적인 문법을 반영한다. 이 부분에는 여러 가지 다양한 변형이 존재하지만, 여기서 보여준 필드 정도가 합리적이라 할 수 있다.

세 가지 자세한 구문 분석 함수를 전체를 분석하는 한 함수 안에 넣을 것이다. 다음은 자세한 구문 분석 함수가 들어 있는 앞 부분을 보여준다.

In [30]:
import datetime

def access_detail_iter(iterable):
    def parse_request(request):
        words = request.split()
        return words[0], ' '.join(words[1:-1], words[-1])
    def parse_time(ts):
        return datetime.datetime.strptime(ts, "%d/%b/%Y:%H:%M:%S  %z")
    agent_pat = re.compile(r"(?P<product>\S*>)\s+"
                          r"\((?P<system>.*?\)\s*"
                          r"\(?P<platform_details_extensions>.*)")
    def parse_agent(user_agent):
        agent_match = agent_pat.match(user_agent)
        if agent_match:
            return AgentDetails(**agent_match.groupdict())

HTTP 요청, 타임 스탬프, 사용자 에이전트 정보를 처리하기 위한 세 가지 구문 분석을 만들었다. 

이 세 가지 구문 분석을 사용하여 Access 객체로부터 AccessDetails 인스턴스를 만든다. access_detial_iter() 함수의 본문은 다음과 같다.

In [35]:
import urllib
def access_detail_iter(iterable):
    def parse_request(request):
        words = request.split()
        return words[0], ' '.join(words[1:-1], words[-1])
    def parse_time(ts):
        return datetime.datetime.strptime(ts, "%d/%b/%Y:%H:%M:%S  %z")
    agent_pat = re.compile(r"(?P<product>\S*>)\s+"
                          r"\((?P<system>.*?\)\s*"
                          r"\(?P<platform_details_extensions>.*)")
    def parse_agent(user_agent):
        agent_match = agent_pat.match(user_agent)
        if agent_match:
            return AgentDetails(**agent_match.groupdict())
    for access in iterable:
        try:
            meth, uri, protocol = parse_request(access.request)
            yield AccessDetails(access= access,
                               time = parse_time(access.time),
                               method = meth,
                                url = urllib.parse.urlparse(uri),
                                protocol = protocol,
                                referrer = urllib.parse.urlparse(access.referrer),
                                agent = parse_agnet(access.user_agent)
                               )
        except ValueError as e:
            print(e, repr(access))

비스한 디자인 패턴을 앞의 access_iter 함수에서도 사용했다. 입력 객체를 구문 분석한 결과로부터 새로운 객체를 만들었다. 새로 만든 AccessDetails 객체는 원래의 Access 객체를 감싼다. 이러한 기법을 사용하면 변경 불가능한 객체를 사용하는 동시에, 더 자세한 정보를 추가할 수 있다.

이 함수는 기본적으로 Access 객체에서 AccessDetail 객체로 가는 매핑이다. map()을 사용하면 다음과 같이 설계를 바꿀 수 있다는 것을 상상할 수 있다.

In [36]:
def access_detail_iter2(iterable):
    def access_detail_builder(access):
        try:
            meth, uri, protocol = parse_request(access.request)
            return AccessDetails(access= access,
                               time = parse_time(access.time),
                               method = meth,
                                url = urllib.parse.urlparse(uri),
                                protocol = protocol,
                                referrer = urllib.parse.urlparse(access.referrer),
                                agent = parse_agnet(access.user_agent)
                               )
        except ValueError as e:
            print(e, repr(access))  
    return filter(None, map(access_detail_builder, iterable))

AccessDetails 객체 생성을 단일 값을 반환하는 함수로 바꿨다. 그렇게 바꾼 함수를 Access 객체의 반복 가능한 입력 스트림에 적용할 수있다. 이 구조 또한 multiprocessing 모듈의 작동 방식과 잘 들어맞는다.

### 자세한 접근 정보 걸러내기

AccessDetails 객체를 걸러내는 필터를 몇 가지 살펴본다. 첫 번째 관심의 대상이 아닌 수많은 부수적인 파일들을 제외시키는 필터의 모음이다. 두 번째 필터는 분석 함수의 일부이며, 이에 대해서는 다음에 살펴본다.

path_filter() 함수는 세 가지 함수의 조합이다.

1. 빈 경로를 제외한다.
2. 특정 파일 이름들을 제외한다.
3. 주어진 확장자를 가진 파일들을 제외한다.

path_filter()를 최적화한 버전을 다음과 같다.

In [39]:
def path_filter(access_detail_iter):
    name_exclude = {'favicon.ico', 'robots.txt', 'humans.txt', 
                    'crossdomain.xml', '_images', 'search.html', 
                    'genindex.html','searchindex.js', 'modindex.html', 'py-modindex.html'}
    ext_exclude = {'.png', '.js', '.css'}
    
    for detail in access_detail_iter:
        path = detail.url.path.split('/')
        if not any(path):
            continue
        if any(p in name_exclude for p in path):
            continue
        if any(final.endswith(ext) for ext in ext_exclude):
            continue
        yield detail

각각의 AccessDetails 객체에 대해 세 가지 필터를 적용한다. 경로가 비어 있거나 제외해야 하는 이름이 경로에 들어 있거나 확장가가 제외 대상인 경우에는 해당 원소를 조용히 무시한다. 

이 함수는 모든 검사를 명령혈 스타일의 for 루프를 사용해 진행하기 때문에 일종의 최적화라고 할 수 이싿.

각각의 검사를 별도의 일급 계층 필터 함수로 정의하는 형태로 설계를 시작할 수도 있다. 예를 들어, 빈 경로를 처리하는 다음과 같은 함수로부터 시작할 수 있을 것이다.

In [40]:
def non_empty_path(detail):
    path = detail.url.path.split('/')
    return any(path)

이 함수는 오직 경로에 이름이 들어 있는 것만을 보장한다. filter() 함수를 다음과 같이 사용할 수 있다.

In [42]:
filter(non_empty_path, access_detail_iter)

non_excluded_names나 non_excluded_ext 검사도 비슷하게 작성할 수 이싿. filter를 연쇄적으로 호출하는 방식으로 활용할 수 있다.

In [44]:
filter(non_excluded_ext, filter(non_excluded_names, filter(non_empty_path, access_detail_iter)))

### 자세한 접근 정보 분석하기

각각의 AccessDetails 객체를 걸러내고 분석할 때 사용할 수 있는 분석 함수를 두 가지 살펴본다. 첫 번째 함수로, 특정 경로만을 통과시키는 걸러내기 함수를 정의할 것이다. 두 번째 함수는 가각의 경로에 대한 접근 회수를 정리할 것이다.

작은 걸러내기 함수를 만들고, 이를 내장 filter 함수로 조합하여 AccessDetails에 적용할 것이다. 이렇게 함성한 걸러내기 함수다.

In [45]:
def book_filter(access_detail_iter):
    def book_in_path(detail):
        path = tuple(l for l in detail.url.path.split('/') if l)
        return path[0] == 'book' and len(path) > 1
    return filter(book_in_path, access_detail_iter)

다음은 관심 대상 객체의 개수를 세는 최종 축약 함수다.

In [46]:
from collections import Counter
def reduce_book_total(access_details_iter):
    counts =Counter()
    for detail in access_detail_iter:
        counts[detail.url.path] += 1
    return counts

이 함수는 Counter() 객체를 만들어 AccessDetails 객체에 있는 각 경로의 빈도를 센다. 특정 경로 집합에 집중하기 위해 reduce_total(book_filter(details))라는 방식을 사용할 것이다. 이렇게 하면 주어진 필터를 통과한 원소에 대한 요약 정보만을 제공한다.

### 완전한 분석 프로세스

다음은 로그 파일의 컬렉션을 분석하는 analysis() 함수다.

In [47]:
def analysis(filename):
    details = path_filter(access_detail_iter(access_iter(local_gzip(filename))))
    books = book_filter(details)
    totals = reduce_book_total(books)
    return totals

### 동시 처리에 다중 프로세스 풀을 사용하기

multiprocessing 모듈을 활용하는 좋은 방법은 처리를 위한 Pool 객체를 만들고, 그 풀 안에 있는 여러 프로세스에게 작업을 할당하는 것이다. 그렇게 하면 OS가 다양한 프로세스의 실행을 서로 뒤섞어줄 것이다. 각각의 프로세스에 I/O와 계산이 섞여 있다면, 시스템의 프로세스들이 계속 바쁘게 작업을 수행하도록 보장할 수 있어야 할 것이다. 프로세스가 IO 완료를 기다리고 있다면, 다른 프로세스가 계산을 수행할 수 있다. IO가 완료되면, 그 IO를 기다리던 프로세스는 실행 가능한 상태가 되어 다른 프로세스와 함께 프로세스 시간을 경쟁하게 된다. 

작업을 각각의 프로세스에 할당하는 매핑은 다음과 같다.

In [49]:
import multiprocessing
with multiprocessing.Pool(4) as workers:
    workers.map(analysis, glob.glob(pattern))

풀을 p개의 프로세스로 시작하면, 전체 애플리케이션에는 p+1개의 프로세스가 존재한다. 1개의 부모와 p개의 자식 프로세스가 존재하기 때문이다. 일반적으로, 작업 프로세스 개수는 CPU(또는 코어)의 개수와 같으며, 부모 프로세스는 풀에 있는 자식 프로세스 중 하나와 CPU를 공유하게 된다. 

경우에 따라 CPU 개수보다 더 많은 작업 프로세스를 할당하는 것이 도움이 될 수 있다. 각 프로세스가 IO 위주의 처리를 수행해야 하는 경우가 이에 해당할 것이다. 여러 작업 프로세스가 IO의 완료를 기다릴 수 있다면, 애플리케이션의 실행 시간이 줄어들 수도 있다.

다중 프로세스 풀 객체에는 작업을 할당할 수 있는 맵과 비슷한 함수가 네 가지 있다. 각각은 map(), imap(), imap_unordered(), startmap()이다. 각각의 차이는 자세한 작업 할당 방식이나 결과 수집 방식에 있다.

map은 완료된 결과가 풀 객체에 할당된 순서대로 할당되기 때문에 순서가 유지된다.

imap은 기본적으로 이 방식에서는 반복 가능 객체에 있는 각각의 원소를 다음으로 사용 가능해진 작업 프로세스에게 전달한다. 이 과정에서 더 많은 통신 비용이 들 수 이싿. 이로 인해 크기가 1보다 큰 청크 값을 추천한다.

imap_unordered 방식은 imap과 비슷하지만, 결과의 순서를 유지하지 않는다. 

다음은 앞에서 보여준 매핑 방식을 변형한 프로그램이다.

In [50]:
import multiprocessing
pattern = '*.gz'
combined = Counter()
with multiprocessing.Pool() as workers:
    for result in workers.imap_unordered(analysis, glob.glob(pattern)):
        combined.update(result)

풀에 있는 각 작업 프로세스의 결과를 모두 모으기 위한 Counter() 함수를 만든다. CPU 코어 개수 만큼의 자식 프로세스로 이뤄진 풀을 만들고 그 풀 객체를 컨텍스트 관리자로 사용한다. 

### apply를 사용해 단일 요청 만들기

map 함수의 여러 변종과 더불어 풀에는 작업자 프로세스 하나에게 작업을 할당할 수 있는 apply 메서드도 있다. map() 메서드는 실제로 apply 메서드를 반복 실행하는 for 루프라는 사실을 알 수 있다. 예를 들어, 다음과 같은 명령을 사용할 수도 있다.

In [51]:
list(workers.apply(analysis, f) for f in glob.glob(pattern))

[]

### map_async, start_async, apply_async 사용하기

map startmap apply의 동작은 Pool 객체에 있는 자식 프로세스에게 작업을 할당하고, 결과가 준비되면 그 결과를 수집하는 것이다. 이로 인해 자식 프로세스가 부모 프로세스가 자신의 결과를 수집할 때까지 기다리는 경우가 생길 수 있다. _async()가 붙은 함수들은 자식의 완료를 기다리지 않는다. 이 함수들이 반환하는 객체르 ㄹ사용해 자식 프로세스가 생성하는 결과를 질의할 수 있다.

다음은 map_async메서드를 사용하는 버전이다.

In [53]:
import multiprocessing
pattern = '*.gz'
combined = Counter()
with multiprocessing.Pool() as workers:
    result = workers.map_async(analysis, glob.glob(pattern))
    data = result.get()
    for c in data:
        combined.update(c)

### 더 복잡한 다중 프로세스 구조

큐나 파이프를 사용해 프로세스에서 다른 프로세스로 객체를 전달할 수도 있다. 프로세스 사이에 공유 메모리를 사용할 수도 있다. 또한 저수준의 록을 공유하여 파일과 같은 공유 자원에 대한 접근을 동기화할 수도 있다. 이러한 구조의 대부분에는 여러 작업 프로세스 사이에 공유된 상태를 명시적으로 관리하는 부분이 들어간다. 특히, 록이나 공유메모리를 사용하는 것은 원래부터 명령형 방식이고, 함수형 프로그램이 접근 방법과는 잘 어울리지 않는다.

그러나 큐나 파이프를 함수적인 방법으로 다룰 수 있다. 설계를 생산자와 소비자 함수로 분리하는 것이다. 생산자를 객체를 만드렁 큐에 넣는다. 소비자는 큐에서 객체를 가져와 처리한다. 그리고 중간 결과를 다은 큐에 넣을 것이다. 이러한 시긍로 진행하면 동시에 실행 중인 프로세스의 네트워크가 생기고, 부하는 이러한 여러 프로세스 사이에 분산될 거시앋. pycsp 패키지를 사용하면 프로세스 사이에 큐를 기반으로 메세지를 교환하는 과정을 단순화할 수 있다. 

### concurrent.futures 모듈 사용하기

이 모듈도 데이터를 동시에 실행되는 프로세스나 스레드 풀에 할당하는 방법을 제공한다. 이 모듈의 API는 상대적으로 간단혐, multiprocessing.Pool() 함수의 인터페이스와 여러 가지 측면에서 비슷하다.

concurrent.futures 모듈의 성능은 거의 multiprocessing 모듈과 비슷하다.

### concurrent.futures 스레드 풀 사용하기

concurrent.futures 모듈은 애플리케이션에서 사용할 수 있는 또 다른 실행기를 제공한다. concurrent.futures.ProcessPoolExecutor 객체 대신 ThreadPoolExecutor를 사용할 수 있다. 이렇게 하면 단일 프로세스 안에 스레드의 풀을 만든다.

그 부분을 제외하면 나머지 구문은 ProcessPoolExecutor 객체를 사용할 떄와 동일하다. 하지만 성능은 엄청나게 다르다. 로그 파일 처리는 IO위주의 작업이다. 프로세스의 모든 스레드가 동일한 운영체제 스케줄링 제약 사항을 따른다. 이로 인해 다중 스레드를 사용하는 로그 파일 처리의 성능은 로그 파일을 순차적으로 처리하는 경우과 거의 비슷하다. 

일반적으로, 다중 프로세스를 사용하는 것이 파이썬 애플리케이션에게 가장 적합해 보인다.

### 스레드와 큐 모듈 사용하기

threading 패키지는 명령형 애플리케이션을 만들 떄 도움이 되는 몇 가지 구성 요소를 제공한다. 이 모듈은 함수형 애플리케이션을 작성하는 데 초점에 맞춰져 있지는 않다. queue 모듈에 들어 있는 스레드 안전한 큐를 스레드 사이에 객체를 전달하는 데 사용할 수 있다.

threading 모듈에는 여러 스레드에 작업을 배분할 수있는 기능은 들어 있지 않다. 이 모듈의 API는 함수형 프로그래밍에 잘 들어맞지 않는다. 

작업을 할당하기 위해 map함수를 사용하는 것은 함수형 프로그래밍에서 바라는 것과 잘 들어맞는다. 이에 따라, 동시성 함수형 프로그래밍을 작성하려는 경우 concurrent.futures 모듈을 가장 사용하기 좋은 방식으로 생각하고, 초점을 맞추는 것이 최선이다.

### 동시 처리 설계하기

함수형 프로그래밍의 관점에서 볼 때 map 함수의 개념을 데이터 원소들에게 동시에 적용하는 데에는 세 가지 방법이 있다. 다음 세 가지 방식 중 하나를 사용할 수 있다.

* multiprocessing.Pool
* concurrent.futures.ProcessPoolExecutor
* concurrent.futures.TreadPoolExecutor

이들과의 상호작용은 거의 비슷하다. 세 가지 모두 반복 가능 컬렉션의 원소에 대해 함수를 적용할 수 있는 map 메서드를 제공한다. 이 메서드는 다른 함수형 프로그래밍 기법과 잘 드러맞느다. 하지만 동시 스레드와 동시 프로세스의 특성에 따라 각가의 성능에는 차이가 있다. 

함수형 프로그래밍 기법을 사용하는 경우의 이점은 전체 처리 과정의 각 부분을 매핑으로 정의할 수 있다는 것이다. 이로 인해 최적의 설계를 위해 여러 가지 다른 구조를 고려하는 데 드는 비용이 실용적일 수 있다.

